## Dependencies

In [1]:
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
model_path = '/kaggle/input/101-googleq-a-train-bert-base-radam3/model.h5'
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
TOKENIZER_PATH = BERT_PATH+'/assets/vocab.txt'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(TOKENIZER_PATH, True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

bert_layer = KerasLayer(BERT_PATH, trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

x = GlobalAveragePooling1D()(sequence_output)
x = Dropout(0.2)(x)
output = Dense(N_CLASS, kernel_initializer='glorot_uniform', activation="sigmoid", name="output")(x)

model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

# Make predictions

In [8]:
Y_test = model.predict(X_test)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.934303,0.689350,0.332039,0.315341,0.489320,0.499907,0.698382,0.664626,0.750480,...,0.923304,0.882123,0.549239,0.963083,0.941921,0.835661,0.168032,0.071883,0.733893,0.922780
1,46,0.913195,0.626222,0.011878,0.702695,0.792536,0.851567,0.586883,0.532599,0.038825,...,0.733956,0.971055,0.740945,0.982008,0.984129,0.914864,0.922374,0.083666,0.164656,0.879006
2,70,0.868501,0.698520,0.049348,0.784566,0.906296,0.911114,0.576862,0.577501,0.073300,...,0.931251,0.941559,0.723495,0.975764,0.963129,0.902037,0.073168,0.047844,0.779924,0.914602
3,132,0.851737,0.371259,0.009807,0.614760,0.825085,0.909597,0.559589,0.461689,0.223972,...,0.640680,0.954471,0.731362,0.970189,0.980489,0.911855,0.898416,0.138262,0.335777,0.899197
4,200,0.940827,0.434820,0.167967,0.724107,0.648296,0.765840,0.670425,0.619948,0.100458,...,0.680418,0.884383,0.615347,0.967971,0.970339,0.813553,0.116211,0.073450,0.533397,0.922687


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.883381,0.606420,0.046027,0.689842,0.801344,0.828213,0.595958,0.494292,0.229392,...,0.786237,0.938275,0.705898,0.961211,0.969800,0.889689,0.539732,0.111132,0.509138,0.920618
std,2812.670060,0.045517,0.146781,0.078354,0.122722,0.108547,0.132992,0.062305,0.088289,0.209695,...,0.100873,0.022943,0.054331,0.013167,0.011949,0.034643,0.323270,0.054806,0.246663,0.019113
min,39.000000,0.721978,0.324762,0.003136,0.239874,0.303237,0.200913,0.436785,0.341249,0.004376,...,0.569282,0.836874,0.518394,0.884944,0.922564,0.755122,0.007134,0.009646,0.034566,0.814918
25%,2572.000000,0.854710,0.472158,0.011988,0.616747,0.747205,0.809515,0.551034,0.427125,0.066913,...,0.696614,0.927448,0.670321,0.953324,0.963349,0.869918,0.182531,0.072704,0.309221,0.908523
50%,5093.000000,0.887169,0.608508,0.018289,0.696252,0.802269,0.872990,0.590242,0.472691,0.146960,...,0.793478,0.942789,0.709008,0.963814,0.971525,0.894561,0.639038,0.105746,0.487431,0.922008
75%,7482.000000,0.918443,0.726613,0.038002,0.770104,0.877670,0.905871,0.637154,0.549611,0.335613,...,0.874817,0.954294,0.743664,0.970174,0.978864,0.916072,0.842721,0.143345,0.726583,0.933865
max,9640.000000,0.976443,0.905264,0.611926,0.972101,0.984212,0.982902,0.782842,0.746838,0.850322,...,0.970599,0.980923,0.840037,0.987893,0.990612,0.963150,0.955538,0.314728,0.980322,0.971340
